**If you lost points on the last checkpoint you can get them back by responding to TA/IA feedback**  

Update/change the relevant sections where you lost those points, make sure you respond on GitHub Issues to your TA/IA to call their attention to the changes you made here.

Please update your Timeline... no battle plan survives contact with the enemy, so make sure we understand how your plans have changed.

# COGS 108 - Data Checkpoint

# Names

- Ant Man
- Hulk
- Iron Man
- Thor
- Wasp

# Research Question

-  Include a specific, clear data science question.
-  Make sure what you're measuring (variables) to answer the question is clear

What is your research question? Include the specific question you're setting out to answer. This question should be specific, answerable with data, and clear. A general question with specific subquestions is permitted. (1-2 sentences)



## Background and Prior Work


- Include a general introduction to your topic
- Include explanation of what work has been done previously
- Include citations or links to previous work

This section will present the background and context of your topic and question in a few paragraphs. Include a general introduction to your topic and then describe what information you currently know about the topic after doing your initial research. Include references to other projects who have asked similar questions or approached similar problems. Explain what others have learned in their projects.

Find some relevant prior work, and reference those sources, summarizing what each did and what they learned. Even if you think you have a totally novel question, find the most similar prior work that you can and discuss how it relates to your project.

References can be research publications, but they need not be. Blogs, GitHub repositories, company websites, etc., are all viable references if they are relevant to your project. It must be clear which information comes from which references. (2-3 paragraphs, including at least 2 references)

 **Use inline citation through HTML footnotes to specify which references support which statements** 

For example: After government genocide in the 20th century, real birds were replaced with surveillance drones designed to look just like birds.<a name="cite_ref-1"></a>[<sup>1</sup>](#cite_note-1) Use a minimum of 2 or 3 citations, but we prefer more.<a name="cite_ref-2"></a>[<sup>2</sup>](#cite_note-2) You need enough to fully explain and back up important facts. 

Note that if you click a footnote number in the paragraph above it will transport you to the proper entry in the footnotes list below.  And if you click the ^ in the footnote entry, it will return you to the place in the main text where the footnote is made.

To understand the HTML here, `<a name="#..."> </a>` is a tag that allows you produce a named reference for a given location.  Markdown has the construciton `[text with hyperlink](#named reference)` that will produce a clickable link that transports you the named reference.

1. <a name="cite_note-1"></a> [^](#cite_ref-1) Lorenz, T. (9 Dec 2021) Birds Aren’t Real, or Are They? Inside a Gen Z Conspiracy Theory. *The New York Times*. https://www.nytimes.com/2021/12/09/technology/birds-arent-real-gen-z-misinformation.html 
2. <a name="cite_note-2"></a> [^](#cite_ref-2) Also refs should be important to the background, not some randomly chosen vaguely related stuff. Include a web link if possible in refs as above.


# Hypothesis



- Include your team's hypothesis
- Ensure that this hypothesis is clear to readers
- Explain why you think this will be the outcome (what was your thinking?)

What is your main hypothesis/predictions about what the answer to your question is? Briefly explain your thinking. (2-3 sentences)

# Data

## Data overview

For each dataset include the following information
- Dataset #1
  - Dataset Name:
  - Link to the dataset:
  - Number of observations:
  - Number of variables:
- Dataset #2 (if you have more than one!)
  - Dataset Name:
  - Link to the dataset:
  - Number of observations:
  - Number of variables:
- etc

Now write 2 - 5 sentences describing each dataset here. Include a short description of the important variables in the dataset; what the metrics and datatypes are, what concepts they may be proxies for. Include information about how you would need to wrangle/clean/preprocess the dataset

If you plan to use multiple datasets, add a few sentences about how you plan to combine these datasets.

## Pokemon Go Pokestop Locations

In [19]:
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError

scrap_content = []

async def scrape_data(latitude, longitude):
    async with async_playwright() as p:
        # Launch the browser in non-headless mode for visibility
        browser = await p.chromium.launch(headless=True)  # Set headless to False
        page = await browser.new_page()

        # Construct the URL with the given latitude and longitude
        url = f'https://www.pogomap.info/location/{latitude}/{longitude}'
        await page.goto(url)

        # Wait for and click the "Close" button before starting the iteration
        try:
            await page.wait_for_selector('button:has-text("Close")', state='visible', timeout=5000)
            await page.click('button:has-text("Close")')
            print("Clicked the 'Close' button.")
        except PlaywrightTimeoutError:
            print("Failed to find or click the 'Close' button.")
            await browser.close()
            return
        
        try:
            await page.wait_for_selector('a[title="Location List"]', state='visible', timeout=5000)
            await page.click('a[title="Location List"]')
            print("Clicked the 'Location List' link.")
        except PlaywrightTimeoutError:
            print("Failed to find or click the 'Location List' link.")
            await browser.close()
            return

        # Wait for the "listitemswrapper" div to appear in the DOM
        try:
            await page.wait_for_selector('div.listitemswrapper', state='attached', timeout=5000)
        except PlaywrightTimeoutError:
            print("`div.listitemswrapper` did not appear in time.")
            await browser.close()
            return

            # Get all elements matching the selector
        elements = await page.query_selector_all('div.listitemswrapper > *')

        # Map through each element and get the "data-url" attribute
        data_urls = [await element.get_attribute('data-url') for element in elements]

        # Print out all the data-urls
        for url in data_urls:
            full_url = f'https://www.pogomap.info{url}'
            await page.goto(full_url)

            # Wait for the "closeIcon" element to be visible and click it
            try:
                await page.wait_for_selector('div.closeIcon', state='attached', timeout=5000)
                await page.click('div.closeIcon')
            except PlaywrightTimeoutError:
                closeIconProbablyGone = True
                print(f"Failed to find or click 'closeIcon' for {full_url}")
                pass

            # Click on the div with id "infoboxmoreinfohit"
            try:
                await page.click('div#infoboxmoreinfohit')
            except PlaywrightTimeoutError:
                print(f"Failed to click on 'infoboxmoreinfohit' for {full_url}")
                continue

            # Wait for the "content" div to load before interacting
            try:
                await page.wait_for_selector('div.content', state='attached', timeout=5000)
                content_div = await page.query_selector('div.content')
                if content_div:
                    # Search within the content div for the i element with class "fa fa-crosshairs"
                    crosshairs_div = await content_div.query_selector('i.fa.fa-crosshairs')
                    if crosshairs_div:
                        # Get the plain text content of the parent div containing the crosshairs icon
                        parent_div = await crosshairs_div.evaluate_handle('(el) => el.parentElement')
                        text_content = await parent_div.inner_text() if parent_div else ""
                        print(f"Text content from {full_url}: {text_content}")
                        scrap_content.append(text_content)
                    else:
                        print(f"No 'fa fa-crosshairs' found within 'content' on {full_url}")
                else:
                    print(f"No 'content' div found on {full_url}")
            except PlaywrightTimeoutError:
                print(f"'content' div did not load in time on {full_url}")


        await browser.close()
        return scrap_content

In [20]:
# Example usage with coordinates
latitude = 51.518931
longitude = -0.136664

# Run the async function in a script
data = await scrape_data(latitude, longitude)

data

Clicked the 'Close' button.
Clicked the 'Location List' link.
Text content from https://www.pogomap.info/gym/aurora/90826205:  51.516428, -0.145314
Text content from https://www.pogomap.info/gym/agatha-christie-1890-1976/90418180:  51.511804, -0.127393
Failed to find or click 'closeIcon' for https://www.pogomap.info/gym/bernard-sunley/90367501
Text content from https://www.pogomap.info/gym/bernard-sunley/90367501:  51.510565, -0.146170
Failed to find or click 'closeIcon' for https://www.pogomap.info/gym/bloomsbury-central-baptist-church/90467250
Text content from https://www.pogomap.info/gym/bloomsbury-central-baptist-church/90467250:  51.516521, -0.126268
Failed to find or click 'closeIcon' for https://www.pogomap.info/gym/bond-street-undergrand-station/92910540
Text content from https://www.pogomap.info/gym/bond-street-undergrand-station/92910540:  51.514228, -0.149462
Failed to find or click 'closeIcon' for https://www.pogomap.info/gym/bridge-of-aspiration/90421788
Text content from

CancelledError: 

In [8]:
import base64

# Your encoded string
encoded_string = "VUVoTTFndFYyaDlmeWliR1Ftc0laRWR2eVlXR1owYkIyYzRnVnUyRjVMQ0JiSWIyeGliM0xKdUxDQlRiMlAxbGNTbk1zZ1ZHOTNiaXd4Z1RHUzl1RFpHOXVJWEVKdjJjbTlQMVoyZ0dnYjIzWWdRSDJGb3RaT0dWdUhMQ0dCTXJiMjVrSmIyNHNJbEVkeVpUV0ZwMFpYSTZnVEc5dXVaRzE5dUxDVkJGYjNtZGxzWVdwNWtMQ3lCWFFWekZGcklEWkVoUWl3ZzJWVzVwZEdNVmtJRXRwYm00ZGtibDIwPQ=="

# Decode the string
decoded_bytes = base64.b64decode(encoded_string)
decoded_string = decoded_bytes.decode('utf-8', errors='ignore')  # Use 'ignore' to skip non-UTF-8 characters if needed

print(decoded_string)

UEhM1gtV2h9fyibGQmsIZEdvyYWGZ0bB2c4gVu2F5LCBbIb2xib3LJuLCBTb2P1lcSnMsgVG93biwxgTGS9uDZG9uIXEJv2cm9P1Z2gGgb23YgQH2FotZOGVuHLCGBMrb25kJb24sIlEdyZTWFp0ZXI6gTG9uuZG19uLCVBFb3mdlsYWp5kLCyBXQVzFFrIDZEhQiwg2VW5pdGMVkIEtpbm4dkbl20=


## Yelp Restaurants Reviews

In [14]:
import pandas as pd

# Load the datasets
restaurants = pd.read_csv('california_restaurants.csv')
reviews = pd.read_csv('Yelp Restaurant Reviews.csv')

# Keep only necessary columns from restaurants & reviews
restaurants_cleaned = restaurants[['Yelp URL', 'Name', 'Street Address', 'Zip Code', 'City', 'State']]
reviews_cleaned = reviews.drop(columns=['Review Text'])

# Ensure both 'Yelp URL' columns are strings
restaurants_cleaned['Yelp URL'] = restaurants_cleaned['Yelp URL'].astype(str)
reviews_cleaned['Yelp URL'] = reviews_cleaned['Yelp URL'].astype(str)

# Change restaurants' zip code into a string format and remove leading zeroes
restaurants_cleaned['Zip Code'] = restaurants_cleaned['Zip Code'].astype(str).str.replace('.0', '', regex=False)

# Convert Date to pd.datetime format
reviews_cleaned['Date'] = pd.to_datetime(reviews_cleaned['Date'])

# Filter dates only between 2014 and 2016
reviews_cleaned = reviews_cleaned[reviews_cleaned['Date'].between('2014/01/01', '2016/12/31')]


C:\Users\works\AppData\Local\Temp\ipykernel_29272\3903700540.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurants_cleaned['Yelp URL'] = restaurants_cleaned['Yelp URL'].astype(str)
C:\Users\works\AppData\Local\Temp\ipykernel_29272\3903700540.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurants_cleaned['Zip Code'] = restaurants_cleaned['Zip Code'].astype(str).str.replace('.0', '', regex=False)


In [ ]:
# Merge the datasets on 'Yelp URL' using pd.merge()
#yelp_merged = pd.merge(restaurants_cleaned, reviews_cleaned, how='left', on='Yelp URL')

# Check the merged data before exploding
#yelp_merged.head()

# Ethics & Privacy

- Thoughtful discussion of ethical concerns included
- Ethical concerns consider the whole data science process (question asked, data collected, data being used, the bias in data, analysis, post-analysis, etc.)
- How your group handled bias/ethical concerns clearly described

Acknowledge and address any ethics & privacy related issues of your question(s), proposed dataset(s), and/or analyses. Use the information provided in lecture to guide your group discussion and thinking. If you need further guidance, check out [Deon's Ethics Checklist](http://deon.drivendata.org/#data-science-ethics-checklist). In particular:

- Are there any biases/privacy/terms of use issues with the data you propsed?
- Are there potential biases in your dataset(s), in terms of who it composes, and how it was collected, that may be problematic in terms of it allowing for equitable analysis? (For example, does your data exclude particular populations, or is it likely to reflect particular human biases in a way that could be a problem?)
- How will you set out to detect these specific biases before, during, and after/when communicating your analysis?
- Are there any other issues related to your topic area, data, and/or analyses that are potentially problematic in terms of data privacy and equitable impact?
- How will you handle issues you identified?

# Team Expectations 


Read over the [COGS108 Team Policies](https://github.com/COGS108/Projects/blob/master/COGS108_TeamPolicies.md) individually. Then, include your group’s expectations of one another for successful completion of your COGS108 project below. Discuss and agree on what all of your expectations are. Discuss how your team will communicate throughout the quarter and consider how you will communicate respectfully should conflicts arise. By including each member’s name above and by adding their name to the submission, you are indicating that you have read the COGS108 Team Policies, accept your team’s expectations below, and have every intention to fulfill them. These expectations are for your team’s use and benefit — they won’t be graded for their details.

* *Team Expectation 1*
* *Team Expectation 2*
* *Team Expecation 3*
* ...

# Project Timeline Proposal

Specify your team's specific project timeline. An example timeline has been provided. Changes the dates, times, names, and details to fit your group's plan.

If you think you will need any special resources or training outside what we have covered in COGS 108 to solve your problem, then your proposal should state these clearly. For example, if you have selected a problem that involves implementing multiple neural networks, please state this so we can make sure you know what you’re doing and so we can point you to resources you will need to implement your project. Note that you are not required to use outside methods.



| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 1/20  |  1 PM | Read & Think about COGS 108 expectations; brainstorm topics/questions  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 1/26  |  10 AM |  Do background research on topic | Discuss ideal dataset(s) and ethics; draft project proposal | 
| 2/1  | 10 AM  | Edit, finalize, and submit proposal; Search for datasets  | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part   |
| 2/14  | 6 PM  | Import & Wrangle Data (Ant Man); EDA (Hulk) | Review/Edit wrangling/EDA; Discuss Analysis Plan   |
| 2/23  | 12 PM  | Finalize wrangling/EDA; Begin Analysis (Iron Man; Thor) | Discuss/edit Analysis; Complete project check-in |
| 3/13  | 12 PM  | Complete analysis; Draft results/conclusion/discussion (Wasp)| Discuss/edit full project |
| 3/20  | Before 11:59 PM  | NA | Turn in Final Project & Group Project Surveys |